In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from src.utils.model.retrieval_model import RetrievalModel

In [2]:
val_rate = 0.2
test_rate = 0.1
batch_size = 100
embedding_dimension = 50
learning_rate = 0.1
early_stopping_flg = True
tensorboard_flg = False
max_epoch_num = 20

In [3]:
behaviors_df = pd.read_csv(
    "data/RentalProperties/user_activity.csv", names=("item_id", "user_id", "event_type", "create_timestamp")
)

In [4]:
# behaviors_df

In [5]:
seen_df = behaviors_df.query('event_type == "seen"')
count_df = pd.DataFrame(seen_df["user_id"].value_counts()).reset_index().rename(columns={"index": "user_id", "user_id": "count"})

unique_user_ids = list(count_df.query("count >= 10")["user_id"])
seen_df = seen_df[seen_df["user_id"].isin(unique_user_ids)]

In [6]:
seen_in_list_df = (
    behaviors_df.query('event_type == "seen_in_list"')
    .groupby(["user_id", "item_id"])
    .size()
    .sort_values(ascending=False)
    .reset_index(name="count")
)

In [7]:
train_val_df, test_df = train_test_split(seen_df, test_size=0.1, stratify=seen_df["user_id"], random_state=1)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df["user_id"], random_state=1)

In [8]:
# バッチサイズで割り切れるように丸める
step_size = int(len(train_df) / batch_size)
train_df = train_df[: step_size * batch_size]

In [9]:
# train_df

In [10]:
print(len(train_df["user_id"].unique()))
print(len(val_df["user_id"].unique()))
print(len(test_df["user_id"].unique()))

2435
2435
2435


In [11]:
train_ratings = tf.data.Dataset.from_tensor_slices({"user_id": train_df["user_id"], "item_id": train_df["item_id"]})
val_ratings = tf.data.Dataset.from_tensor_slices({"user_id": val_df["user_id"], "item_id": val_df["item_id"]})
test_ratings = tf.data.Dataset.from_tensor_slices({"user_id": test_df["user_id"], "item_id": test_df["item_id"]})

2022-10-10 09:26:34.801665: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
train = train_ratings.batch(batch_size)
val = val_ratings.batch(batch_size)
test = test_ratings.batch(batch_size)

In [13]:
unique_user_ids = np.array(
    list((set(train_df["user_id"].unique()) | set(val_df["user_id"].unique()) | set(test_df["user_id"].unique())))
)
unique_item_ids = np.array(
    list(set(train_df["item_id"].unique()) | set(val_df["item_id"].unique()) | set(test_df["item_id"].unique()))
)
unique_item_dataset = tf.data.Dataset.from_tensor_slices(unique_item_ids)

In [14]:
user_id2seen_items = {}
seen_user_ids = list(seen_in_list_df["user_id"].unique())
for seen_user_id in tqdm(seen_user_ids):
    user_id2seen_items[seen_user_id] = []
    seen_items = seen_in_list_df.query(f'user_id == "{seen_user_id}"')
    for i, item in seen_items.iterrows():
        user_id2seen_items[seen_user_id].append({"item_id": item["item_id"], "count": item["count"]})

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3893/3893 [00:35<00:00, 108.16it/s]


In [15]:
import time

item_weights = []
# for batch in train.take(10):
# for batch in tqdm(train.take(100)):
for batch in tqdm(train):
    # そのバッチに含まれるuser_idとitem_id達
    user_ids = batch["user_id"].numpy()
    item_ids = batch["item_id"].numpy()

    item_weights_by_batch = []
    for i, user_id in enumerate(user_ids):
        # 基本のweightsは1にする
        weights = np.ones(len(item_ids), dtype="float32")

        decoded_user_id = user_id.decode("utf-8")
        # もしそのユーザーのviewログがあるアイテムがあり、かつそのアイテムがバッチの中に存在して、ユーザーがクリックしていなかったら、weightを上げる
        if decoded_user_id in user_id2seen_items:
            seen_items = user_id2seen_items[decoded_user_id]
            # 各seen_itemがバッチの中に存在するか？
            for seen_item in seen_items:
                for j, item_id in enumerate(item_ids):
                    decoded_item_id = item_id.decode("utf-8")
                    if seen_item["item_id"] == decoded_item_id and i != j:
                        # weights[j] = seen_item["count"] + 1
                        weights[j] = seen_item["count"]

        item_weights_by_batch.append(weights)
    item_weights.append(item_weights_by_batch)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 406/406 [00:52<00:00,  7.67it/s]


In [16]:
item_weights = np.array(item_weights)
item_weights.shape

(406, 100, 100)

In [17]:
item_weights.dtype

dtype('float32')

In [18]:
item_weights.reshape([step_size * batch_size, batch_size])

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [19]:
train_ratings = tf.data.Dataset.from_tensor_slices(
    {
        "user_id": train_df["user_id"],
        "item_id": train_df["item_id"],
        "item_weights": item_weights.reshape([step_size * batch_size, batch_size]),
    }
)
train2 = train_ratings.batch(batch_size)

In [20]:
indexes = np.where(item_weights == 2)
# item_weightが2になっているインデックスに相当するユーザーとアイテムが、本当にseen_in_list_dfにあるかどうか検査
for i, j, k in zip(indexes[0], indexes[1], indexes[2]):
    if i == 0:
        for batch in train2.take(1):
            user_ids = batch["user_id"].numpy()
            item_ids = batch["item_id"].numpy()
            user_id = user_ids[j].decode("utf-8")
            item_id = item_ids[k].decode("utf-8")

            result = seen_in_list_df.query(f'user_id == "{user_id}" and item_id == "{item_id}"')
            if len(result) == 0:
                print("zero")
            else:
                # print('ok')
                pass
# item_weightの対角成分が2になっていないことを確認
for i, j, k in zip(indexes[0], indexes[1], indexes[2]):
    if j == k:
        print("NG")
        
# item_weightsの内容と、trainから出てくる内容が同一であることをチェック
for i, batch in enumerate(train2):
    if (item_weights[i] != batch["item_weights"]).numpy().all():
        print("NG")

In [21]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = RetrievalModel(
        unique_user_ids=unique_user_ids,
        unique_item_ids=unique_item_ids,
        user_dict_key="user_id",
        item_dict_key="item_id",
        embedding_dimension=embedding_dimension,
        metrics_candidate_dataset=unique_item_dataset,
        # num_hard_negatives=0,
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM),
        # loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM),
        # loss=Customloss(),
        # loss=custom_loss_function,
    )
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


/Users/haruka/dev/jaist-recsys-thema/.venv/lib/python3.9/site-packages/numpy/core/numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [22]:
callbacks = []
if early_stopping_flg:
    callbacks.append(
        tf.keras.callbacks.EarlyStopping(
            monitor="val_total_loss",
            min_delta=0,
            patience=3,
            verbose=0,
            mode="auto",
            baseline=None,
            restore_best_weights=False,
        )
    )
if tensorboard_flg:
    tfb_log_path = log_path + datetime.now().strftime("%Y%m%d-%H%M%S")
    callbacks.append(
        tf.keras.callbacks.TensorBoard(
            log_dir=tfb_log_path,
            histogram_freq=1,
        )
    )

In [23]:
# model.fit(x=train, validation_data=val, epochs=max_epoch_num, callbacks=callbacks)
model.fit(x=train2, validation_data=val, epochs=max_epoch_num, callbacks=callbacks)

2022-10-10 09:28:04.373262: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_FLOAT
      type: DT_STRING
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 40600
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:8"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
        dim {
          size: 100
        }
      }
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {

Epoch 1/20
397/406 [============================>.] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 22.8042 - regularization_loss: 0.0000e+00 - total_loss: 22.8042

2022-10-10 09:28:07.251842: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_STRING
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 10168
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:1"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_STRING
        }
      }
      args {
      

406/406 [==============================] - 9s 20ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 22.5775 - regularization_loss: 0.0000e+00 - total_loss: 22.5775 - val_factorized_top_k/top_1_categorical_accuracy: 4.9174e-04 - val_factorized_top_k/top_5_categorical_accuracy: 0.0016 - val_factorized_top_k/top_10_categorical_accuracy: 0.0026 - val_factorized_top_k/top_50_categorical_accuracy: 0.0070 - val_factorized_top_k/top_100_categorical_accuracy: 0.0099 - val_loss: 15.4017 - val_regularization_loss: 0.0000e+00 - val_total_loss: 15.4017
Epoch 2/20
406/406 [==============================] - 7s 18ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_

2022-10-10 09:28:36.659942: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


406/406 [==============================] - 8s 19ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 12.2547 - regularization_loss: 0.0000e+00 - total_loss: 12.2547 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0032 - val_factorized_top_k/top_10_categorical_accuracy: 0.0065 - val_factorized_top_k/top_50_categorical_accuracy: 0.0222 - val_factorized_top_k/top_100_categorical_accuracy: 0.0295 - val_loss: 14.7961 - val_regularization_loss: 0.0000e+00 - val_total_loss: 14.7961
Epoch 6/20
406/406 [==============================] - 8s 19ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_

In [24]:
model.evaluate(test, return_dict=True)

 1/57 [..............................] - ETA: 10s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0100 - factorized_top_k/top_10_categorical_accuracy: 0.0200 - factorized_top_k/top_50_categorical_accuracy: 0.0300 - factorized_top_k/top_100_categorical_accuracy: 0.0600 - loss: 20.4209 - regularization_loss: 0.0000e+00 - total_loss: 20.4209

2022-10-10 09:29:01.446957: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_STRING
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 5649
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:2"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_STRING
        }
      }
      args {
       

57/57 [==============================] - 4s 73ms/step - factorized_top_k/top_1_categorical_accuracy: 1.7702e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0042 - factorized_top_k/top_50_categorical_accuracy: 0.0198 - factorized_top_k/top_100_categorical_accuracy: 0.0281 - loss: 20.0856 - regularization_loss: 0.0000e+00 - total_loss: 20.0856


{'factorized_top_k/top_1_categorical_accuracy': 0.0001770224771462381,
 'factorized_top_k/top_5_categorical_accuracy': 0.0015932023525238037,
 'factorized_top_k/top_10_categorical_accuracy': 0.004248539451509714,
 'factorized_top_k/top_50_categorical_accuracy': 0.01982651837170124,
 'factorized_top_k/top_100_categorical_accuracy': 0.028146574273705482,
 'loss': 19.177644729614258,
 'regularization_loss': 0,
 'total_loss': 19.177644729614258}

In [25]:
# behaviors_df.query('event_type == "seen" or event_type == "seen_in_list"').groupby(["user_id", "item_id"]).size().sort_values(
#     ascending=False
# ).reset_index(name="count").query("count > 1")
# seen_in_list_items = set(behaviors_df.query('event_type == "seen_in_list"')["item_id"].unique())
# seen_items = set(behaviors_df.query('event_type == "seen"')["item_id"].unique())
# どのユーザーがどのアイテムをseen_in_listしてたか。ここにある組み合わせが0として入ってきたらlossを増やしてあげたい。
# behaviors_df.query('event_type == "seen_in_list"')

# behaviors_df.query('event_type == "seen"')["user_id"].value_counts()

In [26]:
# sample_weightsを事前に作っておく
# sample_weights = []
# # for batch in train.take(10):
# # for batch in tqdm(train.take(100)):
# for batch in tqdm(train):
#     # そのバッチに含まれるuser_idとitem_id達
#     user_ids = batch["user_id"].numpy()
#     item_ids = batch["item_id"].numpy()

#     sample_weights_by_batch = []
#     for i, user_id in enumerate(user_ids):
#         # 基本のweightsは1にする
#         weights = np.ones(len(item_ids))
#         # weights = 1

#         decoded_user_id = user_id.decode("utf-8")
#         # もしそのユーザーのviewログがあるアイテムがあり、かつそのアイテムがバッチの中に存在して、ユーザーがクリックしていなかったら、weightを上げる
#         if decoded_user_id in user_id2seen_items:
#             seen_items = user_id2seen_items[decoded_user_id]
#             # 各seen_itemがバッチの中に存在するか？
#             for seen_item in seen_items:
#                 for j, item_id in enumerate(item_ids):
#                     decoded_item_id = item_id.decode("utf-8")
#                     if seen_item["item_id"] == decoded_item_id and i != j:
#                         weights[j] = seen_item["count"] + 1

#         sample_weights_by_batch.append(weights)
#         # sample_weights.append(weights)
#     sample_weights.append(sample_weights_by_batch)

In [27]:
input = np.array([[7, 7, 7, 7], [7, 7, 7, 7], [7, 7, 7, 7], [7, 7, 7, 7]])  # Input shape: (2, 3, 4)
diagonal = np.array([1, 1, 1, 1])  # Diagonal shape: (2, 3)

In [28]:
tf.linalg.set_diag(input, diagonal)

<tf.Tensor: shape=(4, 4), dtype=int64, numpy=
array([[1, 7, 7, 7],
       [7, 1, 7, 7],
       [7, 7, 1, 7],
       [7, 7, 7, 1]])>

In [29]:
tf.zeros(10)

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [30]:
aaa = []
for i in range(3):
    aaa.append(np.identity(5))

In [31]:
test = np.array(aaa)
test.shape

(3, 5, 5)

In [32]:
test

array([[[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]]])

In [33]:
test.reshape([15, 5])

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [34]:
import tensorflow as tf


class CustomLoss(tf.keras.losses.Loss):
    def __call__(self, y_true, y_pred, sample_weight=None):
        target = tf.convert_to_tensor(y_true)
        output = tf.convert_to_tensor(y_pred)

        output, from_logits = _get_logits(output, from_logits, "Sigmoid", "binary_crossentropy")
        # if from_logits:
        # return tf.nn.sigmoid_cross_entropy_with_logits(labels=target, logits=output)

        epsilon_ = _constant_to_tensor(epsilon(), output.dtype.base_dtype)
        output = tf.clip_by_value(output, epsilon_, 1.0 - epsilon_)

        # Compute cross entropy from probabilities.
        bce = target * tf.math.log(output + epsilon())
        bce += (1 - target) * tf.math.log(1 - output + epsilon())
        return -bce


def custom_loss_function(y_true, y_pred, sample_weight=None):
    # aaa = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM)

    # return aaa(y_true, y_pred)
    loss = tf.reduce_mean(tf.math.abs((y_true - y_pred) ** 3))
    return loss

In [35]:
def rankL(np_rank):
    r = int(np_rank[-1])
    _l = 0
    for k in range(1, r + 1):
        _l += 1.0 / k
    return np.float32(_l)


"""
labels are assumed to be 1 hot encoded
"""


def warp_loss(labels, logits, sample_weights=None):
    # for easy broadcasting
    labels, logits = tf.transpose(labels, [1, 0]), tf.transpose(logits, [1, 0])
    f_y = tf.reduce_sum(logits * labels, axis=0)
    rank = tf.reduce_sum(tf.maximum(tf.sign(1 + logits - f_y), 0), axis=0)
    diff = tf.reduce_sum(tf.maximum(1 + logits - f_y, 0), axis=0)
    with tf.control_dependencies([tf.assert_greater(rank, tf.zeros_like(rank))]):
        return tf.py_func(rankL, [rank], tf.float32) * diff / rank